In [37]:
%reload_ext autoreload
%autoreload 2
from pr3_utils import *
from stereo import *
from helpers import *
from ekf import *
from visualize import *
filename = "./data/10.npz"
t,features,linear_velocity,angular_velocity,K,b,imu_T_cam = load_data(filename)
#Sample every 4th feature
everyth=4
features = features[:,0:-1:everyth,:]


In [38]:
#Initialize Classes
mxlandmarks = features.shape[1]
numLandmarks = mxlandmarks
stereo = stereo(K,b,imu_T_cam, numLandmarks)
ekf = ekf(stereo, numLandmarks)

In [39]:
#Initialize parameters
bigPose = np.zeros((4,4,len(t[0])))
bigPose[:,:,0] = ekf.IMUpose
Wdef = np.diag([0.3,0.3,0.3,0.05,0.05,0.05])

In [62]:
for i in range(1,len(t[0])):
    print(i)
    linvel = linear_velocity[:,i]
    angvel = angular_velocity[:,i]
    tau = t[0,i]-t[0,i-1]

    #Prediction
    ekf.IMUpose = getNextPose(angvel, linvel,  tau, ekf.IMUpose)
    ekf.IMUsig = getNextSig(angvel,linvel, tau, Wdef, ekf.IMUsig)

    #Update
    fs = features[:,:,i]
    fs=fs.T
    validIdxs = (fs[:,0] >0)
    fs=fs.T
    landmarkposeInIMU = stereo.getPoints2(fs)
    landmarkposeInW = ekf.IMUpose @ landmarkposeInIMU #u
    zpred = stereo.getPredZ(landmarkposeInW, ekf.IMUpose)
    z = zpred[:,validIdxs].reshape(-1,1,order='F') #4Nt by 1
    # get H
    H = stereo.getH(ekf.IMUpose, landmarkposeInW)


    break





1
HIHIHIHIHIHIHIHIHIHIHIHIHIHIHIHIHIHIHIHI
(3322, 4, 3)


/Users/john/Desktop/UCSD_mac/ece276a/ECE276A_PR3/code/stereo.py:43: RuntimeWarning: divide by zero encountered in true_divide
  z = self.fsub / (ul-ur)
/Users/john/Desktop/UCSD_mac/ece276a/ECE276A_PR3/code/stereo.py:51: RuntimeWarning: invalid value encountered in matmul
  sb = self.T@sw
